In [ ]:
import numpy as np
import bosonic_qiskit
import qiskit
import random
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.metrics import roc_auc_score
import cmath
import sys

# --- 1. Autograd Engine (Value Class) ---
class Value:
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        self._backward = lambda: None
        self._prev = set(_children)

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out
    
    def __neg__(self):
        return self * Value(-1)

    def log(self):
        val = self.data
        # Clip to prevent log(0) = -inf
        if val < 1e-7: val = 1e-7
        if val > 1.0 - 1e-7: val = 1.0 - 1e-7
        
        out = Value(np.log(val), (self,), 'log')
        def _backward():
            self.grad += (1.0 / val) * out.grad
        out._backward = _backward
        return out
    

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
        
    def tanh(self):
        # Tanh activation allows negative values (Phase Space!)
        x = self.data
        t = (np.exp(2*x) - 1)/(np.exp(2*x) + 1)
        out = Value(t, (self,), 'tanh')
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad = 1
        for v in reversed(topo):
            v._backward()
            
    def zero_grad(self):
        self.grad = 0

# --- 2. Improved Quantum Op (Using a Custom Observable) ---
class QuantumOp(Value):
    def __init__(self, circuit_builder, weights, inputs, cutoff_dim=4):
        self.weights = weights
        self.inputs = inputs
        self.circuit_builder = circuit_builder
        self.cutoff_dim = cutoff_dim
        
        # --- Define the observable for our HYBRID classification task ---
        cv_dim = self.cutoff_dim ** 2
        
        o_cv = np.zeros((cv_dim, cv_dim))
        fraud_idx = 1
        normal_idx = self.cutoff_dim
        o_cv[fraud_idx, fraud_idx] = 1.0
        o_cv[normal_idx, normal_idx] = -1.0
        
        i_qubit = np.identity(2)
        self.observable = np.kron(i_qubit, o_cv)
        
        parents = tuple(weights + inputs)
        
        w_vals = [w.data for w in self.weights]
        i_vals = [i.data for i in self.inputs]
        val = self._run_simulation(w_vals, i_vals)
        super().__init__(val, parents, 'QuantumHybrid')

    def _run_simulation(self, w_vals, i_vals):
        qmr = bosonic_qiskit.QumodeRegister(
            num_qumodes=2,
            num_qubits_per_qumode=self.cutoff_dim
        )
        qr = qiskit.QuantumRegister(1)  # ancilla only

        circ = bosonic_qiskit.CVCircuit(qmr, qr)
        self.circuit_builder(circ, w_vals, i_vals, qmr, qr)

        try:
            qstate, _, _ = bosonic_qiskit.util.simulate(
                circ,
                add_save_statevector=True
            )

            probs = qstate.probabilities_dict()

            n_q = self.cutoff_dim
            p_fraud = 0.0

            for bitstring, prob in probs.items():
                # ---- split registers ----
                cv_bits = bitstring[:2 * n_q]   # CV modes only
                ancilla_bits = bitstring[2 * n_q:]

                # ---- decode Fock occupations ----
                n0 = int(cv_bits[:n_q], 2)
                n1 = int(cv_bits[n_q:], 2)

                # ---- FRAUD DEFINITION ----
                if (n0, n1) in {(1, 0), (0, 1)}:
                    p_fraud += float(prob)

            return max(p_fraud, 1e-6)

        except Exception as e:
            print(f"Simulation failed: {e}")
            return 0.0




    def _backward(self):
        h = 0.001 
        w_numerics = [w.data for w in self.weights]
        i_numerics = [i.data for i in self.inputs]
        for idx, w in enumerate(self.weights):
            w_copy = w_numerics.copy()
            w_copy[idx] += h
            out_plus = self._run_simulation(w_copy, i_numerics)
            w_copy[idx] -= 2*h
            out_minus = self._run_simulation(w_copy, i_numerics)
            grad = (out_plus - out_minus) / (2*h)
            w.grad += self.grad * grad
        for idx, inp in enumerate(self.inputs):
            i_copy = i_numerics.copy()
            i_copy[idx] += h
            out_plus = self._run_simulation(w_numerics, i_copy)
            i_copy[idx] -= 2*h
            out_minus = self._run_simulation(w_numerics, i_copy)
            grad = (out_plus - out_minus) / (2*h)
            inp.grad += self.grad * grad

            # print(f"Grad for input {idx} with data={inp.data:.4f} is {inp.grad:.4f}")
            


# --- 3. Hybrid Model with Tanh & Strong Init ---
class LinearLayer:
    def __init__(self, n_in, n_out):
        self.weights = [[Value(random.gauss(0, 0.2)) for _ in range(n_out)] for _ in range(n_in)]
        self.bias = [Value(0.1) for _ in range(n_out)]
        
    def __call__(self, x):
        out = []
        for j in range(len(self.bias)):
            act = self.bias[j]
            for i in range(len(x)):
                act = act + x[i] * self.weights[i][j]
            out.append(act)
        return out
    
    def parameters(self):
        return [p for row in self.weights for p in row] + self.bias

class HybridFraudDetector:
    def __init__(self, n_features):
        self.c_layer1 = LinearLayer(n_features, 20)
        self.c_layer2 = LinearLayer(20, 14) 
        
        self.q_weights = []
        for i in range(21):
            val = random.uniform(-0.6, 0.6) 
            self.q_weights.append(Value(val))
            
    def forward(self, x_raw):
        x_vals = [Value(xi) for xi in x_raw]
        h1 = self.c_layer1(x_vals)
        h1_act = [h.tanh() for h in h1]
        encoding_params = self.c_layer2(h1_act) 
        out = QuantumOp(self.quantum_circuit, self.q_weights, encoding_params)
        return out
    
    @staticmethod
    def quantum_circuit(circuit, weights, inputs, qmr, qr):
        def to_complex(mag, phase):
            return mag * cmath.exp(1j * phase)

        sq_0 = to_complex(inputs[0], inputs[1])
        sq_1 = to_complex(inputs[2], inputs[3])
        circuit.cv_sq(sq_0, qmr[0])
        circuit.cv_sq(sq_1, qmr[1])
        circuit.cv_bs(inputs[4], qmr[0], qmr[1])
        circuit.cv_r(inputs[6], qmr[0])
        circuit.cv_r(inputs[7], qmr[1])
        d_0 = to_complex(inputs[8], inputs[9])
        d_1 = to_complex(inputs[10], inputs[11])
        circuit.cv_d(d_0, qmr[0])
        circuit.cv_d(d_1, qmr[1])
        
        circuit.cv_bs(weights[0], qmr[0], qmr[1])
        circuit.cv_r(weights[2], qmr[0])
        circuit.cv_r(weights[3], qmr[1])
        s_var_0 = to_complex(weights[4], weights[5])
        s_var_1 = to_complex(weights[6], weights[7])
        circuit.cv_sq(s_var_0, qmr[0])
        circuit.cv_sq(s_var_1, qmr[1])
        circuit.cv_bs(weights[8], qmr[0], qmr[1])
        circuit.cv_r(weights[10], qmr[0])
        circuit.cv_r(weights[11], qmr[1])
        d_var_0 = to_complex(weights[12], weights[13])
        d_var_1 = to_complex(weights[14], weights[15])
        circuit.cv_d(d_var_0, qmr[0])
        circuit.cv_d(d_var_1, qmr[1])
        
        circuit.h(qr[0])
        circuit.ry(weights[18], qr[0])
        
        cd_alpha = to_complex(weights[19], weights[20])
        circuit.cv_c_d(cd_alpha, [qmr[0]], qr[0])
        
    def parameters(self):
        return self.c_layer1.parameters() + self.c_layer2.parameters() + self.q_weights
    
# --- 4. Training - MODIFIED FOR STABILITY ---
def get_full_data(n_samples=1000):
    try:
        df = pd.read_csv(r"D:\Academic\QML_Intern\Anomaly_Detection\Fraud_Detection\creditcard_data.csv")
    except FileNotFoundError:
        print("ERROR: Please update the path to 'creditcard_data.csv'")
        return None, None, None, None
        
    fraud = df[df['Class'] == 1]
    normal = df[df['Class'] == 0]
    n_per = n_samples // 2
    fraud = resample(fraud, n_samples=n_per, random_state=42)
    normal = resample(normal, n_samples=n_per, random_state=42)
    df_bal = pd.concat([fraud, normal])
    
    X = df_bal.drop(['Class', 'Time'], axis=1).values
    y = df_bal['Class'].values
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Setup
X_train, X_test, y_train, y_test = get_full_data(n_samples=10000)
if X_train is not None:
    model = HybridFraudDetector(n_features=29)
    optimizer_params = model.parameters()
    epochs = 10
    
    # --- CHANGE 1: Lower the learning rate ---
    # Cross-entropy can produce large gradients, so a smaller LR is needed for stability.
    lr = 0.001 
    
    print(f"Training with BINARY CROSS-ENTROPY loss...")
    
    for epoch in range(epochs):
        epoch_loss = 0
        start = time.time()
        indices = np.random.permutation(len(X_train))
        
        for i in range(0, len(X_train), 64):
            batch_idx = indices[i:i+64]
            batch_loss = Value(0)
            
            for idx in batch_idx:
                # pred is now a probability p in [0, 1]
                p = model.forward(X_train[idx])
                y_target = y_train[idx]
                # print(f"  Sample {idx}: Predicted p={p.data:.4f}, True y={y_target}", file=sys.stderr)
                
                # --- CHANGE FOR CROSS-ENTROPY ---
                # Loss = -(y*log(p) + (1-y)*log(1-p))
                y = Value(y_target)
                anomaly_weight = 10  # You can tune this hyperparameter
                term = (Value(anomaly_weight) * y * p.log() + (Value(1) + y.__neg__()) * (Value(1) + p.__neg__()).log()).__neg__()
                
                batch_loss = batch_loss + term
                
            batch_loss = batch_loss * Value(1.0/64)
            
            # Zero Grads & Backprop
            for p in optimizer_params: p.zero_grad()
            batch_loss.backward()

            for p in optimizer_params:
                if np.isnan(p.grad):
                    print(f"NaN detected in gradients for parameter with data={p.data}")
                    p.grad = 0.0  # Reset NaN gradients to zero to prevent instability
                else:
                    print(f"Grad for param with data={p.data:.4f} is {p.grad:.4f}")

            # ---- GRADIENT DIAGNOSTICS ----
            max_grad = 0.0
            mean_grad = 0.0
            nonzero = 0

            for p in optimizer_params:
                g = abs(p.grad)
                max_grad = max(max_grad, g)
                mean_grad += g
                if g > 1e-8:
                    nonzero += 1

            mean_grad /= len(optimizer_params)

            # print(
            #     f"\n[GRAD CHECK] "
            #     f"max={max_grad:.2e}, "
            #     f"mean={mean_grad:.2e}, "
            #     f"nonzero={nonzero}/{len(optimizer_params)}"
            # )



            
            # --- CHANGE 2: Implement Gradient Clipping ---
            # This prevents exploding gradients from destabilizing the training.
            clip_value = 1.0 # Do not allow any single gradient to be larger than this.
            for p in optimizer_params:
                p.grad = np.clip(p.grad, -clip_value, clip_value)

            # Update weights
            for p in optimizer_params:
                p.data -= lr * p.grad
                
            epoch_loss += batch_loss.data
            print(f"\rBatch {i//64} | Loss: {batch_loss.data:.4f}", end="")
            
        print(f"\nEpoch {epoch+1} | Avg Loss: {epoch_loss / (len(X_train)/64):.4f} | Time: {time.time()-start:.1f}s")
        
        # Test AUC (still works with probability outputs)
        preds = [model.forward(x).data for x in X_test]
        try:
            print(f"Test AUC: {roc_auc_score(y_test, preds):.4f}")
        except: pass


Training with BINARY CROSS-ENTROPY loss...


c:\Users\PARAM\Documents\Gemini-Test\DDP\boson_env\lib\site-packages\scipy\sparse\_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csc_array is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])


Grad for param with data=0.0721 is 0.0000
Grad for param with data=0.0310 is 0.0000
Grad for param with data=-0.2961 is 0.0000
Grad for param with data=-0.3114 is 0.0000
Grad for param with data=0.2475 is 0.0000
Grad for param with data=0.3234 is 0.0000
Grad for param with data=-0.3852 is 0.0000
Grad for param with data=-0.1142 is 0.0000
Grad for param with data=-0.1993 is 0.0000
Grad for param with data=0.1408 is 0.0000
Grad for param with data=0.2043 is 0.0000
Grad for param with data=0.2862 is 0.0000
Grad for param with data=0.4075 is 0.0000
Grad for param with data=0.4127 is 0.0000
Grad for param with data=0.1961 is 0.0000
Grad for param with data=-0.0859 is 0.0000
Grad for param with data=-0.0635 is 0.0000
Grad for param with data=0.1819 is 0.0000
Grad for param with data=0.2500 is 0.0000
Grad for param with data=0.1289 is 0.0000
Grad for param with data=0.0310 is 0.0000
Grad for param with data=-0.6501 is 0.0000
Grad for param with data=-0.0683 is 0.0000
Grad for param with data=

In [3]:
import bosonic_qiskit
import numpy as np
import random
import cmath
import qiskit
import qiskit.visualization

In [5]:
qmr = bosonic_qiskit.QumodeRegister(1, num_qubits_per_qumode = 6)
qbr = qiskit.QuantumRegister(1)
cr = qiskit.ClassicalRegister(1)

#convenient labeling
qbit = qbr[0]
qumode = qmr[0]

#The circuit is initalized to |0>|0> by default
circuit = bosonic_qiskit.CVCircuit(qmr, qbr, cr)

#put the qubit into a superposition and then execute a controlled displacement of the cavity
alpha = 2
circuit.h(qbr[0])
circuit.cv_c_d(alpha, qmr[0], qbr[0])

#apply a rotation around an angle pi/4
theta = np.pi/4
circuit.cv_r(theta, qmr[0])

#measure qubit in x basis to collapse into even or odd cat state
circuit.measure_x(qbr, cr)

#simulate and plot
try:
    state, _, _ = bosonic_qiskit.util.simulate(circuit)
    print(state)

except Exception as e:
    print(f"Simulation failed: {e}")

Statevector([ 1.91360898e-01-1.17174756e-17j,
             -2.61244889e-17+1.00693886e-17j,
             -1.74079391e-16+5.41250356e-01j,
             -3.08231244e-17-4.97668717e-17j,
             -6.24982077e-01+1.14807345e-16j,
              1.64707854e-17-4.41893493e-17j,
             -5.17175745e-16-4.56422376e-01j,
              2.43676057e-17+2.02942457e-17j,
              2.43968022e-01+3.58680600e-16j,
             -4.84403915e-18+1.03262037e-17j,
             -1.44934297e-16+1.02865950e-01j,
             -1.76719327e-18-1.44154894e-18j,
             -3.58133269e-02+7.89677849e-17j,
              6.30314420e-19-1.27136053e-18j,
             -5.20163420e-18-1.06186416e-02j,
              3.64024128e-19+1.95411955e-19j,
              2.74172148e-03-1.51093820e-18j,
             -6.88508178e-20+6.92360035e-20j,
              2.61120811e-18+6.26935141e-04j,
             -2.11207490e-20-1.76369363e-20j,
             -1.28644376e-04-3.70387478e-19j,
              1.97741955e-21-2.267

In [6]:
qmr = bosonic_qiskit.QumodeRegister(2, num_qubits_per_qumode = 3)
circuit = bosonic_qiskit.CVCircuit(qmr)

circuit.cv_sq2(1, qmr[0], qmr[1])

_, result, fock_counts = bosonic_qiskit.util.simulate(circuit)

In [7]:
def quantum_circuit(circuit, weights, inputs, qmr, qr):
        def to_complex(mag, phase):
            return mag * cmath.exp(1j * phase)

        sq_0 = to_complex(inputs[0], inputs[1])
        sq_1 = to_complex(inputs[2], inputs[3])
        circuit.cv_sq(sq_0, qmr[0])
        circuit.cv_sq(sq_1, qmr[1])
        circuit.cv_bs(inputs[4], qmr[0], qmr[1])
        circuit.cv_r(inputs[6], qmr[0])
        circuit.cv_r(inputs[7], qmr[1])
        d_0 = to_complex(inputs[8], inputs[9])
        d_1 = to_complex(inputs[10], inputs[11])
        circuit.cv_d(d_0, qmr[0])
        circuit.cv_d(d_1, qmr[1])
        
        circuit.cv_bs(weights[0], qmr[0], qmr[1])
        circuit.cv_r(weights[2], qmr[0])
        circuit.cv_r(weights[3], qmr[1])
        s_var_0 = to_complex(weights[4], weights[5])
        s_var_1 = to_complex(weights[6], weights[7])
        circuit.cv_sq(s_var_0, qmr[0])
        circuit.cv_sq(s_var_1, qmr[1])
        circuit.cv_bs(weights[8], qmr[0], qmr[1])
        circuit.cv_r(weights[10], qmr[0])
        circuit.cv_r(weights[11], qmr[1])
        d_var_0 = to_complex(weights[12], weights[13])
        d_var_1 = to_complex(weights[14], weights[15])
        circuit.cv_d(d_var_0, qmr[0])
        circuit.cv_d(d_var_1, qmr[1])
        
        circuit.h(qr[0])
        circuit.ry(weights[18], qr[0])
        
        cd_alpha = to_complex(weights[19], weights[20])
        circuit.cv_c_d(cd_alpha, [qmr[0]], qr[0])

In [8]:
def runsimulation(weights, inputs):
        qmr = bosonic_qiskit.QumodeRegister(
            num_qumodes=2,
            num_qubits_per_qumode=1
        )
        qr = qiskit.QuantumRegister(1)  # ancilla only

        circ = bosonic_qiskit.CVCircuit(qmr, qr)
        
        def to_complex(mag, phase):
            return mag * cmath.exp(1j * phase)

        sq_0 = to_complex(inputs[0], inputs[1])
        sq_1 = to_complex(inputs[2], inputs[3])
        circ.cv_sq(sq_0, qmr[0])
        circ.cv_sq(sq_1, qmr[1])
        circ.cv_bs(inputs[4], qmr[0], qmr[1])
        circ.cv_r(inputs[6], qmr[0])
        circ.cv_r(inputs[7], qmr[1])
        d_0 = to_complex(inputs[8], inputs[9])
        d_1 = to_complex(inputs[10], inputs[11])
        circ.cv_d(d_0, qmr[0])
        circ.cv_d(d_1, qmr[1])
        
        circ.cv_bs(weights[0], qmr[0], qmr[1])
        circ.cv_r(weights[2], qmr[0])
        circ.cv_r(weights[3], qmr[1])
        s_var_0 = to_complex(weights[4], weights[5])
        s_var_1 = to_complex(weights[6], weights[7])
        circ.cv_sq(s_var_0, qmr[0])
        circ.cv_sq(s_var_1, qmr[1])
        circ.cv_bs(weights[8], qmr[0], qmr[1])
        circ.cv_r(weights[10], qmr[0])
        circ.cv_r(weights[11], qmr[1])
        d_var_0 = to_complex(weights[12], weights[13])
        d_var_1 = to_complex(weights[14], weights[15])
        circ.cv_d(d_var_0, qmr[0])
        circ.cv_d(d_var_1, qmr[1])
        
        circ.h(qr[0])
        circ.ry(weights[18], qr[0])
        
        cd_alpha = to_complex(weights[19], weights[20])
        circ.cv_c_d(cd_alpha, [qmr[0]], qr[0])

        try:
            qstate, _, _ = bosonic_qiskit.util.simulate(
                circ,
                add_save_statevector=True
            )

            probs = qstate.probabilities_dict()
            print(probs)
            n_q = 1
            p_fraud = 0.0

            for bitstring, prob in probs.items():
                # ---- split registers ----
                cv_bits = bitstring[:2 * n_q]   # CV modes only
                ancilla_bits = bitstring[2 * n_q:]

                # ---- decode Fock occupations ----
                n0 = int(cv_bits[:n_q], 2)
                n1 = int(cv_bits[n_q:], 2)

                # ---- FRAUD DEFINITION ----
                if (n0, n1) in {(1, 0), (0, 1)}:
                    p_fraud += float(prob)

            return max(p_fraud, 1e-6)

        except Exception as e:
            print(f"Simulation failed: {e}")
            return 0.0

In [14]:
inputs = random.sample(range(1, 100), 12)
weights = [random.gauss(0, 0.2) for i in range(21)]


In [17]:
weights[4] += 0.05
out1 = runsimulation(weights, inputs)

weights[4] -= 0.05
out2 = runsimulation(weights, inputs)

grad = (out1 - out2) / 0.1

print(out1, out2)
print(f"Estimated gradient for weight[4]: {grad:.4f}")

{np.str_('000'): np.float64(0.02458436031400804), np.str_('001'): np.float64(0.056344676945928776), np.str_('010'): np.float64(0.14919974068390357), np.str_('011'): np.float64(0.34592626597335446), np.str_('100'): np.float64(0.006659415011343808), np.str_('101'): np.float64(0.05289992312704189), np.str_('110'): np.float64(0.038093176591379625), np.str_('111'): np.float64(0.3262924413530299)}
{np.str_('000'): np.float64(0.02458436031400804), np.str_('001'): np.float64(0.056344676945928776), np.str_('010'): np.float64(0.14919974068390357), np.str_('011'): np.float64(0.34592626597335446), np.str_('100'): np.float64(0.006659415011343808), np.str_('101'): np.float64(0.05289992312704189), np.str_('110'): np.float64(0.038093176591379625), np.str_('111'): np.float64(0.3262924413530299)}
0.5546853447956437 0.5546853447956437
Estimated gradient for weight[4]: 0.0000
